In [1]:
%run style_and_packages.ipynb

# Beispiel: Rayleigh-Quotient an einem Sechsmassenschwinger beim Ersatzkraftverfahren
## Aufgabenstellung

Das Beispiel ist aus @Dupraz2004 Seite 77 entnommen. Die Lastermittlung wird übernommen. Es wird vom vereinfachten Modell in @fig-ekv_6_modell ausgegangen. Der Fokus liegt auf der Bestimmung der Grundfrequenz.

![Vereinfachtes Modell für einen Mehrmassenschwinger](bilder/aufgabe_ekv_sieben_biege.svg){#fig-ekv_6_modell}

Gesucht:

- Erstelle die Nachgiebigkeitsmatrix (Verwende das Bildungsgesetz)
- Grundschwingzeit

Gegeben:




In [2]:
H_i, m_i,  I, I_x, I_y, f= sp.symbols('H_i, m_i  I  I_x I_y f')
n = 6 # n-Massenschwinger
F = sp.MatrixSymbol('F', n,1)
u = sp.MatrixSymbol('u', n,1)
f_hat = sp.MatrixSymbol('\hat{f}', n,n)
M = sp.MatrixSymbol('M', n,1)


In [3]:
params={
    E: 27*10**9*unit.N/ unit.m**2,
    H_i: 3.105*unit.m,
    m_i: 1278*10**3*unit.N*unit.second**2 /unit.m,
    I_x: 14.89*unit.m**4,
    I_y: 28.27*unit.m**4,
    g : 10 *unit.m /unit.second**2
}

params_plot = convert.param_value(params)

render.dict_to_table(params)

|   |   |
|---|---|
| $E = \frac{27000000000 \text{N}}{\text{m}^{2}}$ | $H_{i} = 3.105 \text{m}$ |
| $I_{x} = 14.89 \text{m}^{4}$ | $I_{y} = 28.27 \text{m}^{4}$ |
| $g = \frac{10 \text{m}}{\text{s}^{2}}$ | $m_{i} = \frac{1278000 \text{N} \text{s}^{2}}{\text{m}}$ |


{{< pagebreak >}}

## Musterlösung

### Rayleigh-Quotient

In [4]:
rl_eqs={
    
}

In [5]:
def nachgiebigkeitsmatrix_nach_bildungsgesetz(n):
    """
    Erstellt die Nachgiebigkeitsmatrix nach dem Bildungsgesetz. Dieses ist nur zulässig für gleichmässige Stockwerkshöhen, sowie gleichbleibenden Stockwerkssteifigkeiten.
    Dies muss abschliessend mit h^3/(6*E*I) mutlipliziert werden.
    """
    from sympy import symbols, zeros

    
    def bildungsgesetz(i,j):
        return j**2*(3*i-j)

    matrix = sp.zeros(n, n)
    matrix_symbols =sp.zeros(n,n)
    matrix_unsymm = sp.zeros(n, n)
    
    for i in range(1,n+1):
        for j in range(1,n+1):
            if i >=j:
                matrix[i-1,j-1] = bildungsgesetz(i,j)
                matrix_symbols[i-1,j-1] = sp.Symbol(f'f_{i,j}')
                matrix_unsymm[i-1,j-1] = bildungsgesetz(i,j)

            if j>i:
                matrix[i-1, j-1] = bildungsgesetz(j,i)
                matrix_symbols[i-1,j-1] = sp.Symbol(f'f_{i,j}')

    
    return matrix, matrix_symbols, matrix_unsymm
            


In [6]:
rl_eqs[f_hat] = sp.UnevaluatedExpr(H_i**3/(6*E*I))*nachgiebigkeitsmatrix_nach_bildungsgesetz(n)[0]

rl_eqs[F] = sp.Matrix(list(range(1, n+1)))
rl_eqs[u] = f_hat * F

In [7]:
rl_eqs[M] = sp.ones(n,1)*m_i


In [8]:
rl_eqs[f] = sp.Mul(sp.UnevaluatedExpr(1 / (2*sp.pi)),sp.sqrt(sp.Sum(sp.HadamardProduct(F, u)[i,0], (i, 0, F.shape[0]-1)) / sp.Sum(sp.HadamardProduct(M,u.applyfunc(lambda x:x**2))[i,0],(i,0,M.shape[0]-1))), evaluate=False)

rl_eqs[T] = 1 / f

Mittels dem Rayleigh-Quotient für das vereinfachte Modell lässt sich die Grundfrequenz direkt bestimmen.

In [9]:
render.eq_display(
    f,rl_eqs[f],
    F, rl_eqs[F],
    M, rl_eqs[M],
    T, rl_eqs[T],
)

                          ____________________________
                         ╱   5                        
                        ╱   ___                       
                       ╱    ╲                         
                      ╱      ╲                        
                     ╱       ╱   (F)[i, 0]⋅(u)[i, 0]  
                    ╱       ╱                         
                   ╱        ‾‾‾                       
     1            ╱        i = 0                      
f = ───⋅         ╱         ────────────────────────── 
    2⋅π         ╱            5                        
               ╱            ___                       
              ╱             ╲                         
             ╱               ╲                2       
            ╱                ╱   (M)[i, 0]⋅(u) [i, 0] 
           ╱                ╱                         
          ╱                 ‾‾‾                       
        ╲╱                 i = 0                      

    ⎡1⎤
    ⎢ ⎥
    ⎢2⎥
    ⎢ ⎥
    ⎢3⎥
F = ⎢ ⎥
    ⎢4⎥
    ⎢ ⎥
    ⎢5⎥
    ⎢ ⎥
    ⎣6⎦

    ⎡mᵢ⎤
    ⎢  ⎥
    ⎢mᵢ⎥
    ⎢  ⎥
    ⎢mᵢ⎥
M = ⎢  ⎥
    ⎢mᵢ⎥
    ⎢  ⎥
    ⎢mᵢ⎥
    ⎢  ⎥
    ⎣mᵢ⎦

    1
T = ─
    f

Dabei entspricht $\mathbf{u}$ dem Verschiebungsvektor infolge des Kraftvektors $\mathbf{F}$. Der Verschiebungsvektor kann mittels Nachgiebigkeitsmatrix bestimmt werden.

#### Nachgiebigkeitsmatrix

Für gleichbleibende Geschosshöhen und Geschosssteifigkeiten lässt sich die Nachgiebigkeitsmatrix leicht mittels dem Bildungsgesetz in @eq-ekv_6_bildungsgesetz ermitteln.

$$
\hat{f}_{i,j} = \frac{H^3}{6EI} \cdot j^2(3i-j) \text{ für } i\geq j
$${#eq-ekv_6_bildungsgesetz}

$\hat{\mathbf{f}}$ entspricht der Nachgiebigkeitsmatrix mit den Einträgen $\hat{f}_{i,j}$.

Beachte dabei, dass die @eq-ekv_6_bildungsgesetz nur für $i\geq j$ gilt. Die Einträge entsprechen folgendem Schema:

In [10]:
display(nachgiebigkeitsmatrix_nach_bildungsgesetz(n)[1])

⎡f_(1, 1)  f_(1, 2)  f_(1, 3)  f_(1, 4)  f_(1, 5)  f_(1, 6)⎤
⎢                                                          ⎥
⎢f_(2, 1)  f_(2, 2)  f_(2, 3)  f_(2, 4)  f_(2, 5)  f_(2, 6)⎥
⎢                                                          ⎥
⎢f_(3, 1)  f_(3, 2)  f_(3, 3)  f_(3, 4)  f_(3, 5)  f_(3, 6)⎥
⎢                                                          ⎥
⎢f_(4, 1)  f_(4, 2)  f_(4, 3)  f_(4, 4)  f_(4, 5)  f_(4, 6)⎥
⎢                                                          ⎥
⎢f_(5, 1)  f_(5, 2)  f_(5, 3)  f_(5, 4)  f_(5, 5)  f_(5, 6)⎥
⎢                                                          ⎥
⎣f_(6, 1)  f_(6, 2)  f_(6, 3)  f_(6, 4)  f_(6, 5)  f_(6, 6)⎦

Unter strikter Anwendung von @eq-ekv_6_bildungsgesetz folgt daraus:

In [11]:
display(nachgiebigkeitsmatrix_nach_bildungsgesetz(n)[2])

⎡2   0    0    0    0    0 ⎤
⎢                          ⎥
⎢5   16   0    0    0    0 ⎥
⎢                          ⎥
⎢8   28  54    0    0    0 ⎥
⎢                          ⎥
⎢11  40  81   128   0    0 ⎥
⎢                          ⎥
⎢14  52  108  176  250   0 ⎥
⎢                          ⎥
⎣17  64  135  224  325  432⎦

Aufgrund von Symmetrie kann diese abschliessend über die Diagonale gespiegelt werden:

In [12]:
render.eq_display(f_hat/sp.UnevaluatedExpr(H_i**3 /(6*E*I)), nachgiebigkeitsmatrix_nach_bildungsgesetz(n)[0])

                ⎡2   5    8   11   14   17 ⎤
                ⎢                          ⎥
                ⎢5   16  28   40   52   64 ⎥
                ⎢                          ⎥
6⋅E⋅I           ⎢8   28  54   81   108  135⎥
─────⋅\hat{f} = ⎢                          ⎥
   3            ⎢11  40  81   128  176  224⎥
 Hᵢ             ⎢                          ⎥
                ⎢14  52  108  176  250  325⎥
                ⎢                          ⎥
                ⎣17  64  135  224  325  432⎦

Durch Multiplikation der *Nachgiebigkeit* mit der *Einwirkung* resultiert die Deformation.

In [13]:
render.eq_display(u, rl_eqs[u],
                 u, calcs.subs_recursive(u, rl_eqs),
                 u, calcs.subs_recursive(u, rl_eqs).doit())

u = \hat{f}⋅F

    ⎡     3         3          3          3          3          3  ⎤    
    ⎢   Hᵢ        Hᵢ         Hᵢ         Hᵢ         Hᵢ         Hᵢ   ⎥    
    ⎢2⋅─────   5⋅─────    8⋅─────   11⋅─────   14⋅─────   17⋅───── ⎥    
    ⎢  6⋅E⋅I     6⋅E⋅I      6⋅E⋅I      6⋅E⋅I      6⋅E⋅I      6⋅E⋅I ⎥    
    ⎢                                                              ⎥    
    ⎢     3          3         3          3          3          3  ⎥    
    ⎢   Hᵢ         Hᵢ        Hᵢ         Hᵢ         Hᵢ         Hᵢ   ⎥    
    ⎢5⋅─────   16⋅─────  28⋅─────   40⋅─────   52⋅─────   64⋅───── ⎥    
    ⎢  6⋅E⋅I      6⋅E⋅I     6⋅E⋅I      6⋅E⋅I      6⋅E⋅I      6⋅E⋅I ⎥    
    ⎢                                                              ⎥ ⎡1⎤
    ⎢     3          3         3          3           3          3 ⎥ ⎢ ⎥
    ⎢   Hᵢ         Hᵢ        Hᵢ         Hᵢ          Hᵢ         Hᵢ  ⎥ ⎢2⎥
    ⎢8⋅─────   28⋅─────  54⋅─────   81⋅─────   108⋅─────  135⋅─────⎥ ⎢ ⎥
    ⎢  6⋅E⋅I      6⋅E⋅I     6⋅E⋅I      6⋅E⋅I       

    ⎡      3 ⎤
    ⎢ 42⋅Hᵢ  ⎥
    ⎢ ────── ⎥
    ⎢  E⋅I   ⎥
    ⎢        ⎥
    ⎢      3 ⎥
    ⎢925⋅Hᵢ  ⎥
    ⎢─────── ⎥
    ⎢ 6⋅E⋅I  ⎥
    ⎢        ⎥
    ⎢      3 ⎥
    ⎢950⋅Hᵢ  ⎥
    ⎢─────── ⎥
    ⎢ 3⋅E⋅I  ⎥
u = ⎢        ⎥
    ⎢       3⎥
    ⎢1535⋅Hᵢ ⎥
    ⎢────────⎥
    ⎢ 3⋅E⋅I  ⎥
    ⎢        ⎥
    ⎢       3⎥
    ⎢2173⋅Hᵢ ⎥
    ⎢────────⎥
    ⎢ 3⋅E⋅I  ⎥
    ⎢        ⎥
    ⎢       3⎥
    ⎢5663⋅Hᵢ ⎥
    ⎢────────⎥
    ⎣ 6⋅E⋅I  ⎦

Durch Einsetzen der bestimmten Verformung in die Gleichung der Eigenfrequenz folgt:

In [14]:
render.eq_display(f, calcs.subs_recursive(f, rl_eqs).doit(),
                 T, calcs.subs_recursive(T, rl_eqs).doit())

                      ________
                     ╱  E⋅I   
    √43665341610⋅   ╱  ────── 
                   ╱     3    
                 ╲╱    Hᵢ ⋅mᵢ 
f = ──────────────────────────
            4993178⋅π         

      √43665341610⋅π  
T = ──────────────────
              ________
             ╱  E⋅I   
    8745⋅   ╱  ────── 
           ╱     3    
         ╲╱    Hᵢ ⋅mᵢ 

### Grundschwingzeit

#### X-Richtung

Es gilt $I$ mit $I_y$ zu substituieren.

In [15]:
u_j_x = calcs.subs_recursive(u, rl_eqs).doit().subs(I, I_y)
f_x = calcs.subs_recursive(f, rl_eqs).doit().subs(I, I_y)
T_x = calcs.subs_recursive(T, rl_eqs).doit().subs(I,I_y)
render.eq_display(I_y, I_y.subs(params),
                  sp.MatrixSymbol('\mathbf{u_x}', n,1), u_j_x.evalf(3),
                  sp.MatrixSymbol('\mathbf{u_x}', n,1), u_j_x.subs(params).evalf(3),
                 )

                 4
I_y = 28.27⋅meter 

               ⎡       3 ⎤
               ⎢42.0⋅Hᵢ  ⎥
               ⎢──────── ⎥
               ⎢ E⋅I_y   ⎥
               ⎢         ⎥
               ⎢        3⎥
               ⎢154.0⋅Hᵢ ⎥
               ⎢─────────⎥
               ⎢  E⋅I_y  ⎥
               ⎢         ⎥
               ⎢        3⎥
               ⎢317.0⋅Hᵢ ⎥
               ⎢─────────⎥
               ⎢  E⋅I_y  ⎥
\mathbf{u_x} = ⎢         ⎥
               ⎢        3⎥
               ⎢512.0⋅Hᵢ ⎥
               ⎢─────────⎥
               ⎢  E⋅I_y  ⎥
               ⎢         ⎥
               ⎢        3⎥
               ⎢724.0⋅Hᵢ ⎥
               ⎢─────────⎥
               ⎢  E⋅I_y  ⎥
               ⎢         ⎥
               ⎢        3⎥
               ⎢944.0⋅Hᵢ ⎥
               ⎢─────────⎥
               ⎣  E⋅I_y  ⎦

               ⎡1.65e-9⋅meter⎤
               ⎢─────────────⎥
               ⎢    newton   ⎥
               ⎢             ⎥
               ⎢6.05e-9⋅meter⎥
               ⎢─────────────⎥
               ⎢    newton   ⎥
               ⎢             ⎥
               ⎢1.24e-8⋅meter⎥
               ⎢─────────────⎥
               ⎢    newton   ⎥
\mathbf{u_x} = ⎢             ⎥
               ⎢2.01e-8⋅meter⎥
               ⎢─────────────⎥
               ⎢    newton   ⎥
               ⎢             ⎥
               ⎢2.84e-8⋅meter⎥
               ⎢─────────────⎥
               ⎢    newton   ⎥
               ⎢             ⎥
               ⎢3.7e-8⋅meter ⎥
               ⎢──────────── ⎥
               ⎣   newton    ⎦

Und für die Grundfrequenz:

In [16]:
render.eq_display('f_x', f_x.subs(params).evalf(3),
                 'T_x', T_x.subs(params).evalf(3))

      1.88 
fₓ = ──────
     second

Tₓ = 0.531⋅second

Nach @SIA261_2020 16.5.2.3

In [22]:
%%capture

# Geschosshöhen
hoehen_floats = []
for i in range(n+1):
    hoehen_floats.append(H_i)
for i in range(len(hoehen_floats)):
    hoehen_floats[i] = np.float64(i*hoehen_floats[i].subs(params_plot))


# Kräfte aus ständigen und quasi ständen Lasten verteilt anhand der Geschosshöhe
F_tot = np.float64(((n*m_i)*g).subs(params_plot))

kraefte_floats = np.zeros(len(hoehen_floats))
for i in range(len(hoehen_floats)):
    kraefte_floats[i] = F_tot*hoehen_floats[i] / np.sum(hoehen_floats)


# Ersatzsteifigkeit fürs FEM
I_y_FEM = np.float64(I_y.subs(params)/unit.m**4)


from ekv_1_FEM import FEM_ersatzstab
Deformation_x_absolut = FEM_ersatzstab(hoehen_floats, kraefte_floats, I_y_FEM)[0]['displacement_absolute']



In [24]:
T_1_x_approx = 2*sp.sqrt(Deformation_x_absolut)*unit.second
render.eq_display('T_1_x_approx', T_1_x_approx.subs(params).evalf(3))

T₁ ₓ ₐₚₚᵣₒₓ = 0.991⋅second

#### Y-Richtung

Es gilt $I$ mit $I_x$ zu substituieren.

In [19]:
u_j_y = calcs.subs_recursive(u, rl_eqs).doit().subs(I, I_x)
f_y = calcs.subs_recursive(f, rl_eqs).doit().subs(I, I_x)
T_y = calcs.subs_recursive(T, rl_eqs).doit().subs(I,I_x)
render.eq_display(I_x, I_x.subs(params),
                  sp.MatrixSymbol('\mathbf{u_y}', n,1), u_j_y.evalf(3),
                  sp.MatrixSymbol('\mathbf{u_y}', n,1), u_j_y.subs(params).evalf(3),
                 )

                4
Iₓ = 14.89⋅meter 

               ⎡       3 ⎤
               ⎢42.0⋅Hᵢ  ⎥
               ⎢──────── ⎥
               ⎢  E⋅Iₓ   ⎥
               ⎢         ⎥
               ⎢        3⎥
               ⎢154.0⋅Hᵢ ⎥
               ⎢─────────⎥
               ⎢   E⋅Iₓ  ⎥
               ⎢         ⎥
               ⎢        3⎥
               ⎢317.0⋅Hᵢ ⎥
               ⎢─────────⎥
               ⎢   E⋅Iₓ  ⎥
\mathbf{u_y} = ⎢         ⎥
               ⎢        3⎥
               ⎢512.0⋅Hᵢ ⎥
               ⎢─────────⎥
               ⎢   E⋅Iₓ  ⎥
               ⎢         ⎥
               ⎢        3⎥
               ⎢724.0⋅Hᵢ ⎥
               ⎢─────────⎥
               ⎢   E⋅Iₓ  ⎥
               ⎢         ⎥
               ⎢        3⎥
               ⎢944.0⋅Hᵢ ⎥
               ⎢─────────⎥
               ⎣   E⋅Iₓ  ⎦

               ⎡3.13e-9⋅meter⎤
               ⎢─────────────⎥
               ⎢    newton   ⎥
               ⎢             ⎥
               ⎢1.15e-8⋅meter⎥
               ⎢─────────────⎥
               ⎢    newton   ⎥
               ⎢             ⎥
               ⎢2.36e-8⋅meter⎥
               ⎢─────────────⎥
               ⎢    newton   ⎥
\mathbf{u_y} = ⎢             ⎥
               ⎢3.81e-8⋅meter⎥
               ⎢─────────────⎥
               ⎢    newton   ⎥
               ⎢             ⎥
               ⎢5.39e-8⋅meter⎥
               ⎢─────────────⎥
               ⎢    newton   ⎥
               ⎢             ⎥
               ⎢7.03e-8⋅meter⎥
               ⎢─────────────⎥
               ⎣    newton   ⎦

Und für die Grundfrequenz:

In [20]:
render.eq_display('f_y', f_y.subs(params).evalf(3),
                 'T_y', T_y.subs(params).evalf(3))

       1.37 
f_y = ──────
      second

T_y = 0.732⋅second

Nach @SIA261_2020 16.5.2.3

In [25]:
I_x_FEM = np.float64(I_x.subs(params)/unit.m**4)


Deformation_y_absolut = FEM_ersatzstab(hoehen_floats, kraefte_floats, I_x_FEM)[0]['displacement_absolute']

In [26]:
T_1_y_approx = 2*sp.sqrt(Deformation_y_absolut)*unit.second
render.eq_display('T_1_y_approx', T_1_y_approx.subs(params).evalf(3))

T_1_y_approx = 1.17⋅second

### Abminderung der Steifigkeit

Um die Rissbildung zu berücksichtigen, wird die Steifigkeit auf 30 % abgemindert.

#### X-Richtung

In [27]:
f_x_red = f_x.subs(E,0.3*E)
T_x_red = T_x.subs(E,0.3*E)

render.eq_display('f_xred', f_x_red.subs(params).evalf(3),
                 'T_x_red', T_x_red.subs(params).evalf(3))

          1.03 
f_xred = ──────
         second

T_x_red = 0.97⋅second

#### Y-Richtung

In [28]:
f_y_red = f_y.subs(E,0.3*E)
T_y_red = T_y.subs(E,0.3*E)
render.eq_display('f_yred', f_y_red.subs(params).evalf(3),
                 'T_y_red', T_y_red.subs(params).evalf(3))

         0.748 
f_yred = ──────
         second

T_y_red = 1.34⋅second